### Unzipping audio files

In [0]:
# %sh
# tar xvzf '/Volumes/unity_poc/adrian/huggingface_cache/openmic-2018-v1.0.0.tgz' -C '/Volumes/unity_poc/adrian/huggingface_cache/'

openmic-2018/
openmic-2018/acknowledgement.md
openmic-2018/audio/
openmic-2018/checksums/
openmic-2018/._class-map.json
openmic-2018/class-map.json
openmic-2018/._license-cc-by.txt
openmic-2018/license-cc-by.txt
openmic-2018/openmic-2018-aggregated-labels.csv
openmic-2018/openmic-2018-individual-responses.csv
openmic-2018/._openmic-2018-metadata.csv
openmic-2018/openmic-2018-metadata.csv
openmic-2018/openmic-2018.npz
openmic-2018/partitions/
openmic-2018/vggish/
openmic-2018/vggish/000/
openmic-2018/vggish/001/
openmic-2018/vggish/002/
openmic-2018/vggish/003/
openmic-2018/vggish/004/
openmic-2018/vggish/005/
openmic-2018/vggish/006/
openmic-2018/vggish/007/
openmic-2018/vggish/008/
openmic-2018/vggish/009/
openmic-2018/vggish/010/
openmic-2018/vggish/011/
openmic-2018/vggish/012/
openmic-2018/vggish/013/
openmic-2018/vggish/014/
openmic-2018/vggish/015/
openmic-2018/vggish/016/
openmic-2018/vggish/017/
openmic-2018/vggish/018/
openmic-2018/vggish/019/
openmic-2018/vggish/020/
openmic-

### Import libraries

In [0]:
! pip install numpy==1.24.0
! pip install librosa
! pip install torch
! pip install torchmetrics
! pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Not uninstalling numpy at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-529e4e9f-6e54-4279-b621-cc7c9892dc0b
    Can't uninstall 'numpy'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
databricks-feature-store 0.14.3 requires pyspark<4,>=3.1.2, which is not installed.
ydata-profiling 4.2.0 requires numpy<1.24,>=1.16.0, but you have numpy 1.24.0 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.0 which is incompatible.
mleap 0.20.0 requires scikit-learn<0.23.0,>=0.22.0, but you have scikit-learn 1.1.1 

In [0]:
import librosa

import os
import numpy as np
import pandas as pd

from datetime import datetime

import glob
import json

import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
from torchmetrics import Precision, Recall, F1Score
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

### Defining helper functions

In [0]:
def split_list_randomly(input_list, fraction):
    # Calculate the number of elements for the first part
    num_elements_first_part = int(len(input_list) * fraction)

    # Shuffle the input list randomly
    shuffled_list = input_list.copy()
    random.shuffle(shuffled_list)

    # Split the shuffled list into two parts
    first_part = shuffled_list[:num_elements_first_part]
    second_part = shuffled_list[num_elements_first_part:]

    return first_part, second_part

In [0]:
def load_json(path):
    """Load the data from a json file."""
    with open(path, "r") as f:
        data = json.load(f)
    return data

In [0]:
def label_probs_tensor(tensor):
    # Get the indices of the max values along each row
    _, indices = torch.max(tensor, dim=1, keepdim=True)
    # Create a tensor of zeros with the same shape as `probs`
    one_hot = torch.zeros_like(tensor)
    # Use scatter_ to set the value of 1 at the max indices
    one_hot.scatter_(1, indices, 1)
    return one_hot

### Defining parameters

In [0]:
# hyperparameters
EPOCHS: int = 100
BATCH_SIZE: int = 64
LEARNING_RATE: float = 1e-03

# regularizers
DROPOUT: int = 0.2
L2_REGULARIZATION: float = 1e-02

# file paths
ROOT: str = "/Volumes/unity_poc/adrian/huggingface_cache"
DATASET_NAME: str = "openmic-2018"
DATA_PATH: str = f"{ROOT}/{DATASET_NAME}"

LABELS_MAP_PATH: str = f"{DATA_PATH}/class-map.json"
AUDIO_DIR: str = f"{DATA_PATH}/audio"
VGGISH_DIR: str = f"{DATA_PATH}/vggish"

PARTITIONS_PATH: str = f"{DATA_PATH}/partitions"
LABELS_PATH: str = f"{DATA_PATH}/openmic-2018-aggregated-labels.csv"

EXPERIMENTS_PATH: str = f'{DATA_PATH}/experiments/'

class_map = load_json(LABELS_MAP_PATH)
NUM_LABELS = len(class_map)
# print(NUM_LABELS)

In [0]:
os.makedirs(EXPERIMENTS_PATH, exist_ok=True)

### Fetch labels 
Utilising exisiting train & test split

In [0]:
# reading the train & test split
split_train = pd.read_csv(f'{PARTITIONS_PATH}/split01_train.csv', header=None)
split_test = pd.read_csv(f'{PARTITIONS_PATH}/split01_test.csv', header=None)

# Load labels
labels_df = pd.read_csv(LABELS_PATH)
labels_df = (labels_df.groupby("sample_key")
                .agg({"instrument": lambda x: x.iloc[np.argmax(x.values)]})
                .reset_index())
labels_df['instrument_label'] = labels_df['instrument'].apply( lambda x: class_map[x])

In [0]:
labels_df['instrument_label'].value_counts().to_dict()

{16: 2488,
 17: 2201,
 18: 1893,
 19: 1564,
 12: 1202,
 13: 1189,
 11: 1147,
 7: 1017,
 9: 974,
 14: 870,
 10: 699,
 3: 673,
 15: 611,
 5: 593,
 6: 585,
 2: 575,
 8: 521,
 0: 511,
 1: 416,
 4: 271}

The data is quite imbalaced. We can incorporate stratified sampling techniques to handle them.

In [0]:
# Get all file paths for vggish embeddings
filepaths = glob.glob(f"{VGGISH_DIR}/*/*.json")

filepaths_df = pd.DataFrame(filepaths, columns=['paths'])
filepaths_df['file_name'] = filepaths_df['paths'].apply( lambda x: x.split('/')[-1])

train_file_names = [sample_key+'.json' for sample_key in split_train[0].values]
test_file_names = [sample_key+'.json' for sample_key in split_test[0].values]
train_file_names, valid_file_names = split_list_randomly(train_file_names, 0.8)

filepaths_df['train'] = filepaths_df['file_name'].apply( lambda x: 1 if x in train_file_names else 0)
filepaths_df['valid'] = filepaths_df['file_name'].apply( lambda x: 1 if x in valid_file_names else 0)
filepaths_df['test'] = filepaths_df['file_name'].apply( lambda x: 1 if x in test_file_names else 0)

train_filepaths = filepaths_df.loc[filepaths_df['train']==1, 'paths'].to_list()
valid_filepaths = filepaths_df.loc[filepaths_df['valid']==1, 'paths'].to_list()
test_filepaths = filepaths_df.loc[filepaths_df['test']==1, 'paths'].to_list()

In [0]:
print('Train\n', filepaths_df['train'].value_counts())
print('Valid\n', filepaths_df['valid'].value_counts())
print('Test\n', filepaths_df['test'].value_counts())

Train
 1    11932
0     8068
Name: train, dtype: int64
Valid
 0    17017
1     2983
Name: valid, dtype: int64
Test
 0    14915
1     5085
Name: test, dtype: int64


### Create new audio features

In [0]:
def extract_features(file):
    """
    Function to extract features from an audio file
    """
    
    # Load the audio file
    y, sr = librosa.load(file)
        
    # Determine if instrument is harmonic or percussive by comparing means
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    harmonic = int(np.mean(y_harmonic) > np.mean(y_percussive))

    # Extracting features from the audio file
    # Mel-frequency cepstral coefficients (MFCCs)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    
    # Get the mel-scaled spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)  
    
    # Compute chroma energy
    chroma = librosa.feature.chroma_cens(y=y, sr=sr)

    # Chromagram from a waveform or power spectrogram
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    
    # Spectral Contrast
    spec_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    
    # Tonnetz (tonal centroid features)
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    
    # Concatenate all features into one array
    features = np.concatenate((
            [harmonic],
            np.mean(mfccs, axis=1),
            np.std(mfccs, axis=1),
            np.mean(spectrogram, axis=1),
            np.std(spectrogram, axis=1),
            np.mean(chroma, axis=1),
            np.std(chroma, axis=1),
            np.mean(chroma_stft, axis=1),
            np.std(chroma_stft, axis=1),
            np.mean(spec_contrast, axis=1),
            np.std(spec_contrast, axis=1),
            np.mean(tonnetz, axis=1),
            np.std(tonnetz, axis=1)
        ))
     
    return features

It took roughly ~4 hours to create the new features for whole corpus, hence avoiding to re-run it again.

In [0]:
# # Initialize a list to store the data
# data = []

# # Define the root directory where your 'audio' folder is located
# root_dir = '/Volumes/unity_poc/adrian/huggingface_cache/openmic-2018/audio'

# # Walk through all the subdirectories and files in the root directory
# for subdir, dirs, files in os.walk(root_dir):
#     for file in files:
#         # Construct the full file path
#         file_path = os.path.join(subdir, file)
#         # Check if the file is an audio file (you can check extensions e.g., .ogg)
#         if file_path.lower().endswith('.ogg'):
#             try:
#                 # Extract features
#                 features = extract_features(file_path)
#                 # Append the features and the file name to the data list
#                 data.append((subdir.split('/')[-1], file.split('.')[0], file_path, features))

#             except Exception as e:
#                 print(f"Error processing {file_path}: {e}")

In [0]:
# audio_df = pd.DataFrame(data, columns=['subfolder', 'file', 'file_path', 'features'])
# audioDF = spark.createDataFrame(audio_df)
# audioDF.write.mode('overwrite').saveAsTable('hive_metastore.ml.openmic_features')

In [0]:
feat_DF = spark.read.table('hive_metastore.ml.openmic_features')
feat_df = feat_DF.toPandas()
feat_df.head(10).display()

subfolder file file_path features 074 074128_69120 /Volumes/unity_poc/adrian/huggingface_cache/openmic-2018/audio/074/074128_69120.ogg List(0.0, -60.630348205566406, 91.29920959472656, 25.779613494873047, 27.657779693603516, -10.486942291259766, 2.9730987548828125, -7.238142967224121, 1.775221824645996, -6.407293796539307, 2.652238368988037, -7.912901401519775, -1.1007782220840454, -18.495277404785156, 45.520660400390625, 22.412336349487305, 20.943891525268555, 12.045969009399414, 9.948504447937012, 9.504117012023926, 6.9817986488342285, 9.383047103881836, 8.867377281188965, 8.719090461730957, 9.561664581298828, 7.668825149536133, 8.119629859924316, 72.09171295166016, 124.43505096435547, 79.0710678100586, 24.402009963989258, 21.450965881347656, 85.85224914550781, 184.02113342285156, 177.8071746826172, 109.45983123779297, 267.83135986328125, 168.91744995117188, 49.24896240234375, 204.16905212402344, 79.43523406982422, 24.295194625854492, 14.927663803100586, 7.806790351867676, 14.464518547058105, 9.912934303283691, 33.509334564208984, 56.653385162353516, 11.50400161743164, 5.056946277618408, 9.355119705200195, 6.134038925170898, 9.85368824005127, 5.6642889976501465, 5.716389179229736, 6.7138237953186035, 7.312931060791016, 15.38885498046875, 8.662915229797363, 2.6910266876220703, 1.3703205585479736, 1.8357219696044922, 3.5922112464904785, 4.222217082977295, 2.285418748855591, 4.492114067077637, 4.954931259155273, 1.605127215385437, 1.5577499866485596, 1.835856556892395, 3.5318708419799805, 3.0855202674865723, 2.065455198287964, 1.97881281375885, 0.9551324844360352, 1.7364802360534668, 0.9064584374427795, 0.948574960231781, 2.041930913925171, 1.4869096279144287, 1.4201864004135132, 1.6292093992233276, 1.6675552129745483, 1.0426441431045532, 0.9255102276802063, 0.9810183644294739, 0.7839269042015076, 1.034539818763733, 1.0751229524612427, 1.3414247035980225, 0.6939886808395386, 0.7096937894821167, 0.7747558951377869, 0.4676392376422882, 0.44102218747138977, 1.0340055227279663, 0.826829195022583, 0.3443913757801056, 0.3839748501777649, 0.5441367626190186, 0.9094080924987793, 0.6036571860313416, 0.6463333964347839, 0.5259955525398254, 0.37263569235801697, 0.9997273683547974, 0.2952997088432312, 0.20370228588581085, 0.24678124487400055, 0.32026198506355286, 0.35122546553611755, 0.3667670488357544, 0.5242208242416382, 0.2819964289665222, 0.23954704403877258, 0.1748075932264328, 0.3006739914417267, 0.29181885719299316, 0.2233024686574936, 0.3804140090942383, 0.31103283166885376, 0.3428744077682495, 0.35708367824554443, 0.33718839287757874, 0.23685084283351898, 0.20324601233005524, 0.13530486822128296, 0.18024541437625885, 0.21361768245697021, 0.19612787663936615, 0.19804809987545013, 0.2911185026168823, 0.2903470993041992, 0.42090994119644165, 0.4498882591724396, 0.3952590823173523, 0.2660285234451294, 0.16909177601337433, 0.11059557646512985, 0.12719275057315826, 0.11063436418771744, 0.10667823255062103, 0.11716032773256302, 0.16615739464759827, 0.1489495187997818, 0.10260697454214096, 0.13522113859653473, 0.11993853747844696, 0.0765928402543068, 0.07412595301866531, 0.06348466873168945, 0.11572568118572235, 0.09596643596887589, 0.20045919716358185, 0.09950525313615799, 327.3939208984375, 531.451171875, 289.62664794921875, 88.80784606933594, 102.14079284667969, 128.6755828857422, 193.91680908203125, 172.02102661132812, 138.56764221191406, 389.5275573730469, 239.54510498046875, 78.11157989501953, 340.11981201171875, 104.93916320800781, 39.425445556640625, 23.774351119995117, 11.887216567993164, 24.796546936035156, 15.007261276245117, 51.92365646362305, 75.37824249267578, 18.31416130065918, 8.529946327209473, 17.06057357788086, 7.799929618835449, 13.167901992797852, 6.637983798980713, 9.94047737121582, 10.589527130126953, 9.928214073181152, 18.425474166870117, 13.67440414428711, 4.517458438873291, 3.064401865005493, 2.989619255065918, 8.861997604370117, 5.086073398590088, 2.446883201599121, 5.511779308319092, 10.519583702087402, 2.80768

In [0]:
# writing labelled dataset for training ML algorithms
filepaths_df['sample_key'] = filepaths_df['file_name'].apply( lambda x: x.split('.')[0])

samples_df = feat_df\
    .merge(labels_df, left_on='file', right_on='sample_key')\
    .merge(filepaths_df[['sample_key', 'train', 'valid', 'test']], on='sample_key')\
    .drop(columns=['sample_key', 'file_path'])

train_df = samples_df[samples_df['train']==1].copy()
valid_df = samples_df[samples_df['valid']==1].copy()
test_df = samples_df[samples_df['test']==1].copy()

spark.createDataFrame(train_df).write.mode('overwrite').saveAsTable('hive_metastore.ml.openmic_train')
spark.createDataFrame(valid_df).write.mode('overwrite').saveAsTable('hive_metastore.ml.openmic_valid')
spark.createDataFrame(test_df).write.mode('overwrite').saveAsTable('hive_metastore.ml.openmic_test')

In [0]:
class OpenmicDataset(Dataset):
    """Dataset for loading VGGish JSON objects."""

    def __init__(self, file_list, labels_df, num_classes, vggish=False, transform=None):
        """
        Args:
            file_list (list): List of file paths for the JSON files.
            labels_df (pd.DataFrame): DataFrame containing the labels for each file. The DataFrame should have a column for file identifiers and a column for labels.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.file_list = file_list
        self.labels_df = labels_df
        self.num_classes = num_classes
        self.vggish = vggish
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        json_path = self.file_list[idx]
        sample_key = os.path.splitext(os.path.basename(json_path))[0]

        # Using VGGish features
        if self.vggish == True:
            with open(json_path, 'r') as f:
                data = json.load(f)
                features = torch.tensor(data['features']).float()
        else:
            # Using newly created features
            features = torch.tensor(feat_df.loc[feat_df['file']==sample_key, 'features'].values[0]).float()

        if self.transform:
            features = self.transform(features)

        # creating label tensor
        label = self.labels_df.loc[self.labels_df['sample_key'] == sample_key, 'instrument_label'].values[0]
        # label_tensor = torch.tensor(label)
        label_tensor = F.one_hot(torch.tensor(label), num_classes=self.num_classes).float()

        return features, label_tensor

def create_data_loader(file_list, labels_df, batch_size=BATCH_SIZE, vggish=False, shuffle=True, transform=None):
    """
    Create a DataLoader for the VGGish dataset.

    Args:
        file_list (list): List of file paths for the JSON files.
        labels_df (pd.DataFrame): DataFrame containing the labels for each file. The DataFrame should have a column for file identifiers and a column for labels.
        batch_size (int): Batch size.
        shuffle (bool): Whether to shuffle the dataset.
        transform (callable, optional): Optional transform to be applied on each sample.

    Returns:
        DataLoader: DataLoader for the VGGish dataset.
    """
    dataset = OpenmicDataset(file_list=file_list, labels_df=labels_df, num_classes=NUM_LABELS, vggish=vggish, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

In [0]:
train_dataloader = create_data_loader(file_list=train_filepaths, labels_df=labels_df, batch_size=BATCH_SIZE, shuffle=True, vggish=False)
valid_dataloader = create_data_loader(file_list=valid_filepaths, labels_df=labels_df, batch_size=BATCH_SIZE, shuffle=True, vggish=False)
test_dataloader = create_data_loader(file_list=test_filepaths, labels_df=labels_df, batch_size=BATCH_SIZE, shuffle=True, vggish=False)

In [0]:
vggish_train_dataloader = create_data_loader(file_list=train_filepaths, labels_df=labels_df, batch_size=BATCH_SIZE, shuffle=True, vggish=True)
vggish_valid_dataloader = create_data_loader(file_list=valid_filepaths, labels_df=labels_df, batch_size=BATCH_SIZE, shuffle=True, vggish=True)
vggish_test_dataloader = create_data_loader(file_list=test_filepaths, labels_df=labels_df, batch_size=BATCH_SIZE, shuffle=True, vggish=True)

In [0]:
# # check if the loaders are working correctly?
# for i, (features, labels) in enumerate(vggish_test_dataloader):
#     print(f"Batch {i+1}")
#     print("Features:", features)
#     print("Labels:", labels)

#     print(features.shape)
#     print(labels.shape)
    
#     # Optionally, break after the first batch to just see one example
#     if i == 0:
#         break

### Modelling

#### I. A simple neural network with 2 hidden layers

##### Model definition

In [0]:
class SimpleNNClassifier(nn.Module):
    def __init__(self, input_shape, hidden_size, num_classes=NUM_LABELS, dropout=DROPOUT, l2_regularization=L2_REGULARIZATION):
        super(SimpleNNClassifier, self).__init__()
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.input_shape = input_shape
        self.dropout = dropout
        
        self.flatten = nn.Flatten()
        if l2_regularization is not None:
            self.dense1 = nn.Linear(input_shape, self.hidden_size, bias=True)
        else:
            self.dense1 = nn.Linear(input_shape, self.hidden_size)
        self.dropout_layer = nn.Dropout(dropout)
        self.dense2 = nn.Linear(self.hidden_size, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.dense1(x) # avoiding using any activation function before dropout layer
        x = self.dropout_layer(x)
        x = F.relu(self.dense2(x))
        x = F.softmax(x, dim=1)
        return x


## sample model check
# model = SimpleNNClassifier(
#     input_shape=10*128, 
#     hidden_size=1024,
#     dropout=DROPOUT,
#     l2_regularization=L2_REGULARIZATION,
#     num_classes=NUM_LABELS)
# print(model)

In [0]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


##### Model trainer definition

In [0]:
class MetricsHistory:
    """Custom callback to track metrics during training."""

    def __init__(self):
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        for metric, value in logs.items():
            if metric not in self.history:
                self.history[metric] = []
            self.history[metric].append(value)

    def save_to_file(self, export_path, filename='metrics.json'):
        filepath = os.path.join(export_path, filename)
        with open(filepath, 'w') as f:
            json.dump(self.history, f)

class ModelTrainer:
    def __init__(self, model, learning_rate=LEARNING_RATE, loss_fn=nn.CrossEntropyLoss(), export_path=EXPERIMENTS_PATH):
        self.model = model.to(device)
        self.learning_rate = learning_rate
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=L2_REGULARIZATION)
        self.loss_fn = loss_fn
        self.metrics = {
            "precision": Precision(task='multiclass', num_classes=NUM_LABELS, average='micro').to(device),
            "recall": Recall(task='multiclass', num_classes=NUM_LABELS, average='micro').to(device),
            "f1": F1Score(task='multiclass', num_classes=NUM_LABELS, average='micro').to(device)
        }
        self.metrics_history = MetricsHistory()

        self.runid = datetime.now().strftime("%Y%m%d-%H%M%S")
        self.export_path = os.path.join(export_path, self.runid)
        os.makedirs(self.export_path, exist_ok=True)

    def save_model(self, filename='model.pth'):
        model_path = os.path.join(self.export_path, filename)
        print(f"==> Saving model to {model_path} ...")
        torch.save(self.model.state_dict(), model_path)

    def train(self, train_loader, epochs=5, validation_loader=None):
        for epoch in range(epochs):
            self.model.train()
            train_loss = 0.0
            # Reset metrics at the start of each epoch
            for metric in self.metrics.values():
                metric.reset()
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, targets)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item() * inputs.size(0)
                # Update metrics
                for metric in self.metrics.values():
                    metric.update(label_probs_tensor(outputs), targets)
            train_loss /= len(train_loader.dataset)
            # Print training metrics
            print_metrics('Train', epoch, train_loss, self.metrics)

            # Update metrics history after training
            epoch_logs = {'train_loss': train_loss}
            for name, metric in self.metrics.items():
                epoch_logs[name] = metric.compute().item()
                metric.reset()  # Make sure to reset metrics after logging
            self.metrics_history.on_epoch_end(epoch, epoch_logs)

            if validation_loader:
                val_loss, val_metrics = self.evaluate(validation_loader)
                val_metrics_string = ', '.join(f'{name}: {value:.4f}' for name, value in val_metrics.items())
                print(f'Validation - Epoch {epoch+1}, Loss: {val_loss}, {val_metrics_string}')

        # Save metrics history after training
        self.metrics_history.save_to_file(self.export_path)

    def evaluate(self, data_loader):
        self.model.eval()
        loss = 0.0
        # Reset metrics
        for metric in self.metrics.values():
            metric.reset()
        with torch.no_grad():
            for inputs, targets in data_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = self.model(inputs)
                loss += self.loss_fn(outputs, targets).item() * inputs.size(0)
                # Update metrics
                for metric in self.metrics.values():
                    metric.update(label_probs_tensor(outputs), targets)
        loss /= len(data_loader.dataset)
        metrics_result = {name: metric.compute().item() for name, metric in self.metrics.items()}
        return loss, metrics_result

def print_metrics(phase, epoch, loss, metrics):
    metrics_result = {name: metric.compute().item() for name, metric in metrics.items()}
    metrics_string = ', '.join(f'{name}: {value:.4f}' for name, value in metrics_result.items())
    print(f"{phase} - Epoch {epoch+1}, Loss: {loss:.4f}, {metrics_string}")

##### Training

In [0]:
model = SimpleNNClassifier(
    input_shape=10*128, 
    hidden_size=1024,
    dropout=0.1,
    l2_regularization=L2_REGULARIZATION,
    num_classes=NUM_LABELS)
print(model)

# Initialize the ModelTrainer
trainer = ModelTrainer(model, LEARNING_RATE, export_path=f'{EXPERIMENTS_PATH}')

# Train the model
trainer.train(vggish_train_dataloader, epochs=3, validation_loader=vggish_valid_dataloader)

# Evaluate the model on the test set
test_loss, test_metrics = trainer.evaluate(vggish_test_dataloader)
test_metrics_string = ', '.join(f'{name}: {value:.4f}' for name, value in test_metrics.items())
print(f'Testing - Loss: {test_loss}, {test_metrics_string}')

# Save the trained model
trainer.save_model()

SimpleNNClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense1): Linear(in_features=1280, out_features=1024, bias=True)
  (dropout_layer): Dropout(p=0.1, inplace=False)
  (dense2): Linear(in_features=1024, out_features=20, bias=True)
)
Train - Epoch 1, Loss: 3.0182, precision: 0.9060, recall: 0.9060, f1: 0.9060
Validation - Epoch 1, Loss: 3.0174770848366625, precision: 0.9061, recall: 0.9061, f1: 0.9061
Train - Epoch 2, Loss: 3.0181, precision: 0.9060, recall: 0.9060, f1: 0.9060
Validation - Epoch 2, Loss: 3.0174771064965467, precision: 0.9061, recall: 0.9061, f1: 0.9061
Train - Epoch 3, Loss: 3.0138, precision: 0.9064, recall: 0.9064, f1: 0.9064
Validation - Epoch 3, Loss: 2.993675564424807, precision: 0.9084, recall: 0.9084, f1: 0.9084
Testing - Loss: 2.9627166913556726, precision: 0.9115, recall: 0.9115, f1: 0.9115
==> Saving model to .\openmic-2018\experiments\20240208-135124\model.pth ...


#### II. A simple neural network with 3 hidden layers

##### Model definition

In [0]:
class NNClassifier(nn.Module):
    def __init__(self, input_shape, hidden_size, num_classes, dropout=DROPOUT, l2_regularization=L2_REGULARIZATION):
        super(NNClassifier, self).__init__()
        self.num_classes = num_classes
        self.hidden_size = hidden_size
        self.input_shape = input_shape
        self.dropout = dropout
        
        self.flatten = nn.Flatten()
        if l2_regularization is not None:
            self.dense1 = nn.Linear(input_shape, self.hidden_size, bias=True)
        else:
            self.dense1 = nn.Linear(input_shape, self.hidden_size)
        self.dropout_layer = nn.Dropout(dropout)
        self.dense2 = nn.Linear(self.hidden_size, (self.hidden_size//2))
        self.dense3 = nn.Linear((self.hidden_size//2), num_classes)

    def forward(self, x):
        x = self.flatten(x) 
        x = torch.tanh(self.dense1(x)) # not using any activation function before dropout layer
        x = self.dropout_layer(x)
        x = F.relu(self.dense2(x))
        x = F.relu(self.dense3(x))
        return F.softmax(x, dim=1)

## sample model check
# model = NNClassifier(
#     input_shape=357, 
#     hidden_size=1024,
#     dropout=DROPOUT,
#     l2_regularization=None,
#     num_classes=NUM_LABELS)
# print(model)

##### Model trainer with early stopping mechanism

In [0]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [0]:
class MetricsHistory:
    """Custom callback to track metrics during training."""

    def __init__(self):
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        for metric, value in logs.items():
            if metric not in self.history:
                self.history[metric] = []
            self.history[metric].append(value)

    def save_to_file(self, export_path, filename='metrics.json'):
        filepath = os.path.join(export_path, filename)
        with open(filepath, 'w') as f:
            json.dump(self.history, f)

# trainer with early stopping
class ModelTrainer:
    def __init__(self, model, learning_rate=LEARNING_RATE, loss_fn=nn.BCEWithLogitsLoss(), export_path=EXPERIMENTS_PATH, patience=5):
        self.model = model.to(device)
        self.learning_rate = learning_rate
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=L2_REGULARIZATION)
        self.loss_fn = loss_fn
        self.metrics = {
            "precision": Precision(task='multiclass', num_classes=NUM_LABELS, average='micro').to(device),
            "recall": Recall(task='multiclass', num_classes=NUM_LABELS, average='micro').to(device),
            "f1": F1Score(task='multiclass', num_classes=NUM_LABELS, average='micro').to(device)
        }
        self.metrics_history = MetricsHistory()

        self.runid = datetime.now().strftime("%Y%m%d-%H%M%S")
        self.export_path = os.path.join(export_path, self.runid)
        os.makedirs(self.export_path, exist_ok=True)

        # Early stopping attributes
        self.patience = patience
        self.best_val_loss = float('inf')
        self.epochs_no_improve = 0

    def save_model(self, filename='model.pth'):
        model_path = os.path.join(self.export_path, filename)
        print(f"==> Saving model to {model_path} ...")
        torch.save(self.model.state_dict(), model_path)

    def train(self, train_loader, epochs=5, validation_loader=None):
        for epoch in range(epochs):
            self.model.train()
            train_loss = 0.0
            # Reset metrics at the start of each epoch
            for metric in self.metrics.values():
                metric.reset()
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, targets)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item() * inputs.size(0)
                # Update metrics
                for metric in self.metrics.values():
                    metric.update(label_probs_tensor(outputs), targets)
            train_loss /= len(train_loader.dataset)
            # Print training metrics
            print_metrics('Train', epoch, train_loss, self.metrics)

            # Save metrics history after training
            epoch_logs = {'train_loss': train_loss}
            for name, metric in self.metrics.items():
                epoch_logs[name] = metric.compute().item()
                metric.reset()  # Make sure to reset metrics after logging
            self.metrics_history.on_epoch_end(epoch, epoch_logs)

            if validation_loader:
                val_loss, val_metrics = self.evaluate(validation_loader)
                val_metrics_string = ', '.join(f'{name}: {value:.4f}' for name, value in val_metrics.items())
                print(f'Validation - Epoch {epoch+1}, Loss: {val_loss}, {val_metrics_string}')

                # Early stopping logic
                if val_loss < self.best_val_loss:
                    self.best_val_loss = val_loss
                    self.epochs_no_improve = 0
                else:
                    self.epochs_no_improve += 1
                    print(f'Validation loss did not improve, patience: {self.epochs_no_improve}/{self.patience}')

                if self.epochs_no_improve >= self.patience:
                    print("Early stopping triggered")
                    self.save_model(filename=f'best_model.pth')  # Save the best model
                    break  # Exit the training loop

        # Save metrics history after training
        self.metrics_history.save_to_file(self.export_path)

    def evaluate(self, data_loader):
        self.model.eval()
        loss = 0.0
        # Reset metrics
        for metric in self.metrics.values():
            metric.reset()
        with torch.no_grad():
            for inputs, targets in data_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = self.model(inputs)
                loss += self.loss_fn(outputs, targets).item() * inputs.size(0)
                # Update metrics
                for metric in self.metrics.values():
                    metric.update(label_probs_tensor(outputs), targets)
        loss /= len(data_loader.dataset)
        metrics_result = {name: metric.compute().item() for name, metric in self.metrics.items()}
        return loss, metrics_result

def print_metrics(phase, epoch, loss, metrics):
    metrics_result = {name: metric.compute().item() for name, metric in metrics.items()}
    metrics_string = ', '.join(f'{name}: {value:.4f}' for name, value in metrics_result.items())
    print(f"{phase} - Epoch {epoch+1}, Loss: {loss:.4f}, {metrics_string}")


##### Training

###### VGGish features

In [0]:
model = NNClassifier(
    input_shape=10*128, 
    hidden_size=1024,
    dropout=0.5,
    l2_regularization=L2_REGULARIZATION,
    num_classes=NUM_LABELS)
print(model)

# Initialize the ModelTrainer
trainer = ModelTrainer(model, LEARNING_RATE, export_path=f'{EXPERIMENTS_PATH}')

# Train the model
trainer.train(vggish_train_dataloader, epochs=EPOCHS, validation_loader=vggish_valid_dataloader)

# Evaluate the model on the test set
test_loss, test_metrics = trainer.evaluate(vggish_test_dataloader)
test_metrics_string = ', '.join(f'{name}: {value:.4f}' for name, value in test_metrics.items())
print(f'Testing - Loss: {test_loss}, {test_metrics_string}')


# # Save the trained model
# trainer.save_model()

NNClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense1): Linear(in_features=1280, out_features=1024, bias=True)
  (dropout_layer): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=1024, out_features=512, bias=True)
  (dense3): Linear(in_features=512, out_features=20, bias=True)
)
Train - Epoch 1, Loss: 0.7156, precision: 0.9122, recall: 0.9122, f1: 0.9122
Validation - Epoch 1, Loss: 0.7156241530146009, precision: 0.9124, recall: 0.9124, f1: 0.9124
Train - Epoch 2, Loss: 0.7154, precision: 0.9122, recall: 0.9122, f1: 0.9122
Validation - Epoch 2, Loss: 0.7158050476965231, precision: 0.9124, recall: 0.9124, f1: 0.9124
Validation loss did not improve, patience: 1/5
Train - Epoch 3, Loss: 0.7159, precision: 0.9122, recall: 0.9122, f1: 0.9122
Validation - Epoch 3, Loss: 0.7159519474297772, precision: 0.9124, recall: 0.9124, f1: 0.9124
Validation loss did not improve, patience: 2/5
Train - Epoch 4, Loss: 0.7160, precision: 0.9121, recall: 0.9121, f1: 0.9121
Validati

###### New audio features

In [0]:
# building model with new features
model = NNClassifier(
    input_shape=357, 
    hidden_size=1024,
    dropout=0.5,
    l2_regularization=L2_REGULARIZATION,
    num_classes=NUM_LABELS)
print(model)

# Initialize the ModelTrainer
trainer = ModelTrainer(model, LEARNING_RATE, export_path=f'{EXPERIMENTS_PATH}')

# Train the model
trainer.train(train_dataloader, epochs=EPOCHS, validation_loader=valid_dataloader)

# Evaluate the model on the test set
test_loss, test_metrics = trainer.evaluate(test_dataloader)
test_metrics_string = ', '.join(f'{name}: {value:.4f}' for name, value in test_metrics.items())
print(f'Testing - Loss: {test_loss}, {test_metrics_string}')


# # Save the trained model
# trainer.save_model()

NNClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense1): Linear(in_features=357, out_features=1024, bias=True)
  (dropout_layer): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=1024, out_features=512, bias=True)
  (dense3): Linear(in_features=512, out_features=20, bias=True)
)
Train - Epoch 1, Loss: 0.7159, precision: 0.9120, recall: 0.9120, f1: 0.9120
Validation - Epoch 1, Loss: 0.715951934481802, precision: 0.9125, recall: 0.9125, f1: 0.9125
Train - Epoch 2, Loss: 0.7160, precision: 0.9122, recall: 0.9122, f1: 0.9122
Validation - Epoch 2, Loss: 0.7159518364128793, precision: 0.9125, recall: 0.9125, f1: 0.9125
Train - Epoch 3, Loss: 0.7160, precision: 0.9120, recall: 0.9120, f1: 0.9120
Validation - Epoch 3, Loss: 0.7159516101230362, precision: 0.9125, recall: 0.9125, f1: 0.9125
Train - Epoch 4, Loss: 0.7160, precision: 0.9122, recall: 0.9122, f1: 0.9122
Validation - Epoch 4, Loss: 0.7159522921496336, precision: 0.9125, recall: 0.9125, f1: 0.9125
Validation